In [1]:
import sys, os
import multiprocessing

logical_cores = os.cpu_count()
physical_cores = multiprocessing.cpu_count()

os.environ["OMP_NUM_THREADS"] = "1"  # set number of OpenMP threads to run in parallel
os.environ["MKL_NUM_THREADS"] = "1"  # set number of MKL threads to run in parallel

import matplotlib.pyplot as plt

from quspin.operators import hamiltonian  # Hamiltonians and operators
from quspin.basis import spin_basis_1d  # Hilbert space spin basis_1d
from quspin.basis.user import user_basis  # Hilbert space user basis
from quspin.basis.user import (
    op_sig_32,
    map_sig_32,
)
from numba import carray, cfunc  # numba helper functions
from numba import uint32, int32  # numba data types

import numpy as np
from scipy.special import comb
np.set_printoptions(precision=24, floatmode='fixed')
from joblib import Parallel, delayed
import time as T
from tqdm import tqdm
from tqdm.notebook import tqdm

def ThreeSpinIsing_GS__no_symm_(N, h):
    if N % 3 != 0:
        raise ValueError("System size N must be a multiple of 3.")
    basis = spin_basis_1d(N, pauli=True)
    three_spin_list = [[-1.0, i, (i+1) % N, (i+2) % N] for i in range(N)]  # Three-spin interaction
    transverse_field_list = [[-h, i] for i in range(N)]  # Transverse field
    static = [["zzz", three_spin_list], ["x", transverse_field_list]]
    dynamic = []  # No time-dependent terms
    H = hamiltonian(static, dynamic, basis=basis, dtype=np.float64, check_symm=False, check_herm=False, check_pcon=False)
    E, psi0 = H.eigsh(k=1, which='SA')  # Smallest eigenvalue ('SA' for smallest algebraic)
    psi0 = np.sign(psi0[0])*psi0
    return E[0], psi0  # Return ground state energy and eigenvector

def D1_bitmask_pattern__3(N):
    pattern = ['1', '1', '0']
    num_cycles = N // len(pattern)
    remaining_elements = N % len(pattern)
    operator_string = pattern * num_cycles + pattern[:remaining_elements]
    return operator_string
def D2_bitmask_pattern__3(N):
    pattern = ['0', '1', '1']
    num_cycles = N // len(pattern)
    remaining_elements = N % len(pattern)
    operator_string = pattern * num_cycles + pattern[:remaining_elements]
    return operator_string
def D3_bitmask_pattern__3(N):
    pattern = ['1', '0', '1']
    num_cycles = N // len(pattern)
    remaining_elements = N % len(pattern)
    operator_string = pattern * num_cycles + pattern[:remaining_elements]
    return operator_string

In [2]:
N = 6

@cfunc(op_sig_32,locals=dict(s=int32, n=int32, b=uint32))
def op(op_struct_ptr, op_str, site_ind, N, args):
    op_struct = carray(op_struct_ptr, 1)[0]
    err = 0
    site_ind = N - site_ind - 1                         # convention for QuSpin for mapping from bits to sites.
    n = (op_struct.state >> site_ind) & 1               # either 0 or 1
    s = (((op_struct.state >> site_ind) & 1) << 1) - 1  # either -1 or 1
    b = 1 << site_ind
    if op_str == 120:                                   # "x" is integer value 120 = ord("x")
        op_struct.state ^= b
    elif op_str == 121:                                 # "y" is integer value 120 = ord("y")
        op_struct.state ^= b
        op_struct.matrix_ele *= 1.0j * s
    elif op_str == 43:                                  # "+" is integer value 43 = ord("+")
        if n:
            op_struct.matrix_ele = 0
        else:
            op_struct.state ^= b                        # create spin
    elif op_str == 45:                                  # "-" is integer value 45 = ord("-")
        if n:
            op_struct.state ^= b                        # destroy spin
        else:
            op_struct.matrix_ele = 0
    elif op_str == 122:                                 # "z" is integer value 120 = ord("z")
        op_struct.matrix_ele *= s
    elif op_str == 110:                                 # "n" is integer value 110 = ord("n")
        op_struct.matrix_ele *= n
    elif op_str == 73:                                  # "I" is integer value 73 = ord("I")
        pass
    else:
        op_struct.matrix_ele = 0
        err = -1
    return err
op_args = np.array([], dtype=np.uint32)
# ---------------------------------------------------------------
#######  define symmetry maps  #######
# ---------------------------------------------------------------
@cfunc(map_sig_32,locals=dict(shift=uint32,xmax=uint32,x1=uint32,x2=uint32,period=int32,l=int32))
def translation(x, N, sign_ptr, args):
    """works for all system sizes N."""
    shift = args[0]                                     # translate state by shift sites
    period = N                                          # periodicity/cyclicity of translation
    xmax = args[1]
    l = (shift + period) % period
    x1 = x >> (period - l)
    x2 = (x << l) & xmax
    return x2 | x1

T_args = np.array([1, (1 << N) - 1], dtype=np.uint32)

D1_bitmask = int(''.join(D1_bitmask_pattern__3(N)), 2)
D2_bitmask = int(''.join(D2_bitmask_pattern__3(N)), 2)
D3_bitmask = int(''.join(D3_bitmask_pattern__3(N)), 2)

@cfunc(map_sig_32)
def D1(x, N, sign_ptr, args):
    """works for all system sizes N."""
    return x ^ D1_bitmask
D1_args = np.array([], dtype=np.uint32)
@cfunc(map_sig_32)
def D2(x, N, sign_ptr, args):
    """works for all system sizes N."""
    return x ^ D2_bitmask
D2_args = np.array([], dtype=np.uint32)
@cfunc(map_sig_32)
def D3(x, N, sign_ptr, args):
    """works for all system sizes N."""
    return x ^ D3_bitmask
D3_args = np.array([], dtype=np.uint32)
maps = dict(
    D1_block=(D1, 2, 0, D1_args),
    D2_block=(D2, 2, 0, D2_args),
    D3_block=(D3, 2, 0, D3_args),     
    T_block=(translation, N, 0, T_args),   
)
op_dict = dict(op=op, op_args=op_args)
basis__ = user_basis(
    np.uint32,
    N,
    op_dict,
    allowed_ops=set("+-xyznI"),
    sps=2,
    Ns_block_est=int((2**(N-2))/(N-1)),
    **maps,)

three_spin_list = [[-1.0, i, (i+1) % N, (i+2) % N] for i in range(N)]    # Three-spin interaction

def ThreeSpinIsing_GS__sublattice_symm_3(h):
    transverse_field_list = [[-h, i] for i in range(N)]                             # Transverse field
    static = [["zzz", three_spin_list], ["x", transverse_field_list]]                                                                    # No time-dependent terms
    H = hamiltonian(static, [], basis=basis__, dtype=np.float64, check_symm=False, check_herm=False, check_pcon=False)
    gs_energy, psi0 = H.eigsh(k=1, which='SA')  
    norm = np.linalg.norm(psi0)
    if norm != 0:
        psi0 /= norm
    return gs_energy[0], np.sign(psi0[0])*psi0

In [3]:
gs_energy, wv_func = ThreeSpinIsing_GS__sublattice_symm_3(0.9)
gs_energy


np.float64(-7.135642302774768)

In [4]:
en, wv = ThreeSpinIsing_GS__sublattice_symm_3(1.0)
en

np.float64(-7.483314773547881)

In [5]:
len(wv)

6